# Importing useful libraries

In [103]:
import pandas as pd
import psycopg2

# Create a connection to the database

In [104]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

# Next use that connect to get a cursor that we will use to execute queries.

In [105]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)

# For this demo we will use automactic commit so that each action is commited without having to call conn.commit() after each command. The ability to rollback and commit transactions are a feature of Relational Databases. 

In [106]:
conn.set_session(autocommit=True)

#  <center>Retial datawarehouse schema design for postgre database</center>
<center>
<img style="float: center;height:400px;" src="Retail_Schema.jpg"><br><br>
</center>

# Creating dim_customer table and inserting records in it

In [107]:
cur.execute("drop table if exists dim_customer")
cur.execute("drop table if exists dim_product")
cur.execute("drop table if exists fact_transactions")
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS dim_customer (\
                customer_id int,\
                dob varchar , \
                gender char , \
                city_code varchar);")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
with open('customer.csv', 'r') as row:
    next(row)# Skip the header row.
    cur.copy_from(row, 'dim_customer', sep=',')

#  Checking referential intergrity   
# try: 
#     cur.execute("insert into customer values(null,'11/04/1991','M',5\
#                );")
# except psycopg2.Error as e: 
#     print("Error: Issue creating table")
#     print (e)

# Creating dim_product table  and inserting records in it

In [72]:
cur.execute("drop table if exists dim_product ")
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS dim_product (\
                prod_cat_code int,\
                prod_cat varchar,\
                prod_sub_cat_code int,\
                prod_subcat varchar\
            );")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
with open('prod_cat_info.csv', 'r') as row:
    next(row)# Skip the header row.
    cur.copy_from(row, 'dim_product', sep=',')

# Creating fact_transactions table and inserting records in it

In [73]:
cur.execute("drop table if exists fact_transactions ")
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS fact_transactions (\
                transaction_id bigint,\
                cust_id int,\
                tran_date varchar(12),\
                prod_subcat_code int,\
                prod_cat_code int,\
                qty int,\
                rate int,\
                tax float,\
                total_amt float,\
                store_type varchar\
                );")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
with open('transactions.csv', 'r') as row:
    next(row)# Skip the header row.
    cur.copy_from(row, 'fact_transactions', sep=',')

# Analysis

In [75]:
# cur.execute("SELECT * FROM pg_catalog.pg_tables where schemaname!='pg_catalog';")
# print(pd.DataFrame(cur.fetchall()))

In [86]:
cur.execute("""select * from fact_transactions limit 5""")
print(cur.fetchall())

[(80712190438, 270351, '28-02-2014', 1, 1, -5, -772, 405.3, -4265.3, 'e-Shop'), (29258453508, 270384, '27-02-2014', 5, 3, -5, -1497, 785.925, -8270.925, 'e-Shop'), (51750724947, 273420, '24-02-2014', 6, 5, -2, -791, 166.11, -1748.11, 'TeleShop'), (93274880719, 271509, '24-02-2014', 11, 6, -3, -1363, 429.345, -4518.345, 'e-Shop'), (51750724947, 273420, '23-02-2014', 6, 5, -2, -791, 166.11, -1748.11, 'TeleShop')]


In [79]:
# What is the total number of rows in each of the three tables
cur.execute("""select 'no of rows in customer table',count(*) from dim_customer
union
select 'no of rows in prod_info table', count(*) from dim_product
union
select 'no of rows in transactions table',count(*) from fact_transactions;""")
print(cur.fetchall())

[('no of rows in prod_info table', 23), ('no of rows in customer table', 5647), ('no of rows in transactions table', 23053)]


In [90]:
# What is the total number of transactions that have a return
cur.execute("""select * from fact_transactions
where qty<0 limit 5""")
print(cur.fetchall())

[(80712190438, 270351, '28-02-2014', 1, 1, -5, -772, 405.3, -4265.3, 'e-Shop'), (29258453508, 270384, '27-02-2014', 5, 3, -5, -1497, 785.925, -8270.925, 'e-Shop'), (51750724947, 273420, '24-02-2014', 6, 5, -2, -791, 166.11, -1748.11, 'TeleShop'), (93274880719, 271509, '24-02-2014', 11, 6, -3, -1363, 429.345, -4518.345, 'e-Shop'), (51750724947, 273420, '23-02-2014', 6, 5, -2, -791, 166.11, -1748.11, 'TeleShop')]


In [94]:
# Which city has max number number of customers
cur.execute("""select city_code,count(*) as total from dim_customer group by city_code
having count(*)=(select max(total) as max_value from (select city_code,count(*) as total 
from dim_customer group by city_code) f);""")
print(cur.fetchall())

[('3', 595)]


In [96]:
# Net total revenue generated in categories Electronics and books
cur.execute("""select b.prod_cat,sum(a.total_amt) as revenue 
from fact_transactions a join dim_product b
on a.prod_cat_code=b.prod_cat_code
where b.prod_cat in ('Books','Electronics')
group by b.prod_cat;""")
print(cur.fetchall())

[('Electronics', 53612318.1750001), ('Books', 76936164.2399989)]


In [98]:
# Total Revenue earned from the Electronics and Clothing from flagship stores
cur.execute("""select b.prod_cat,a.Store_type,sum(a.total_amt) as total_revenue from fact_transactions a 
join dim_product b
on a.prod_cat_code=b.prod_cat_code
where b.prod_cat in ('Clothing','Electronics') and a.Store_type='Flagship store'
group by b.prod_cat,a.Store_type""")
print(cur.fetchall())

[('Clothing', 'Flagship store', 3583269.69000001), ('Electronics', 'Flagship store', 11075680.2)]


In [102]:
#
cur.execute("""select c.prod_subcat,sum(total_amt) as total_revenue
from fact_transactions a 
left join dim_customer b 
on a.cust_id=b.customer_id
left join dim_product c
on a.prod_cat_code=c.prod_cat_code and a.prod_subcat_code=c.prod_sub_cat_code
where b.gender = 'M' and c.prod_cat='Electronics'
group by a.prod_subcat_code,c.prod_subcat;""")
print(cur.fetchall())

[('Mobiles', 1192413.235), ('Computers', 1091417.34), ('Personal Appliances', 1107593.435), ('Cameras', 1172702.245), ('Audio and video', 1138983.17)]
